# Event Account Setup

This notebook configures the Snowflake account as an event account for the native app created by the ACF, using the pickle files generated from the released code. 

**NOTE:** this notebook should only be executed in accounts intended to serve as event accounts, and not in the same account where the ACF and the native app reside. For more information on native app event sharing, visit https://docs.snowflake.com/en/developer-guide/native-apps/event-about.


## PREREQUISITE: Enable GitHub External Access Integration

- If your account ***does not*** have an External Access Integration for the GitHub API, execute the commands below:
```
CREATE OR REPLACE NETWORK RULE gh_network_rule
MODE = EGRESS
TYPE = HOST_PORT
VALUE_LIST = ('github.com', 'api.github.com');

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION gh_access_integration
ALLOWED_NETWORK_RULES = (gh_network_rule)
ENABLED = true;
```
- Once the External Access Integration for the GitHub API as been created (or if one already exists), enable this notebook to use it, by following the steps here: https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-external-access#enable-external-access-integrations-eai.
    - **NOTE:** this step restarts the notebook.



## STEP 1: Initialization

In [ ]:
import base64
import codecs
import os
import pandas as pd
import pickle
import re
import requests
import sqlparse
import streamlit as st

session = get_active_session()

#tag session
session.sql(f"""ALTER SESSION SET QUERY_TAG = '{{"origin":"sf_sit","name":"acf","version":{{"major":1, "minor":7}},"attributes":{{"env":"event_acct,"component":"installer","type":"notebook"}}'""").collect()

#get current_role
current_role = session.get_current_role().replace('"','')

st.success(f"Session initialized for role: {current_role} 🎉")

## STEP 2: Function definition

In [ ]:
def decode(obj):
    pickled = codecs.decode(obj.encode(), 'base64')
    return pickle.loads(pickled)

def get_pickle(path):
    req = requests.get(path)
    if req.status_code == requests.codes.ok:
        req = req.json()
        content = base64.b64decode(req['content']).decode("utf-8")
        return content
    else:
        return 'Content was not found.'

## STEP 3: Set Event Account Parameters

In [ ]:
#get account org and locator
event_account_org = pd.DataFrame(session.sql(f"""SELECT CURRENT_ORGANIZATION_NAME()""").collect()).iloc[0,0]
event_account_locator = session.get_current_account().replace('"','')

acf_app_code = st.text_input("Enter the ACF App Code 👇", help="The unique identifier for your native app (i.e. `ACME`). Use the same identifier when setting up the ACF account.")
acf_acct_locator = st.text_input("Enter the ACF Account Locator 👇", help="The account identifier for the provider's Snowflake where the ACF will be deployed(this can be found by executing the `SELECT CURRENT_ACCOUNT()` command)")
acf_acct_name = st.text_input("Enter the ACF Account Name 👇", help="The account name of the provider's Snowflake where the ACF will be deployed(this can be found by executing the `SELECT CURRENT_ACCOUNT_NAME()` command)")

## STEP 4: Get pickle file and decode

In [ ]:
decoded_event_acct_obj = None

event_acct_pickle = get_pickle('https://api.github.com/repos/Snowflake-Labs/sfguide-application-control-framework/contents/pickles/01_event_acct.pickle')

if event_acct_pickle != 'Content was not found.':
    decoded_event_acct_obj = decode(event_acct_pickle)
    st.success(f"Event Account Setup Pickle Decoded 🎉")    
else:
    print ('Content was not found.')

## STEP 5: Execute commands from decoded pickle file

In [ ]:
verbose = st.selectbox("Verbose?", ("Select...", "Y", "N"), index=0,)

event_account_setup_list = decoded_event_acct_obj['event_account_setup'].items()
for file_name, file_content in event_account_setup_list:
    file_content = str(file_content.decode("utf-8"))
    #replace SnowSQL variables with values from Step 3 and comment out SnowSQL print/set commands
    repl = {"&{APP_CODE}": f"{acf_app_code}"
            ,"&APP_CODE": f"{acf_app_code}"
            ,"&{ORG_NAME}": f"{event_account_org}"
            ,"&ORG_NAME": f"{event_account_org}"
            ,"&{EVENT_ACCOUNT_LOCATOR}": f"{event_account_locator}"
            ,"&EVENT_ACCOUNT_LOCATOR": f"{event_account_locator}"
            ,"&{ACF_ACCOUNT_NAME}": f"{acf_acct_name}"
            ,"&ACF_ACCOUNT_NAME": f"{acf_acct_name}"
            ,"&{ACF_ACCOUNT_LOCATOR}": f"{acf_acct_locator}"
            ,"&ACF_ACCOUNT_LOCATOR": f"{acf_acct_locator}"
            ,"&&&&":"&&"
            ,"!print": "--!print"
            ,"!set": "--!set"
          }

    repl = dict((re.escape(k), v) for k, v in repl.items()) 
    pattern = re.compile("|".join(repl.keys()))
    file_content = pattern.sub(lambda m: repl[re.escape(m.group(0))], file_content)

    #format file_content
    file_content = sqlparse.format(file_content, strip_comments=True).strip()

    #execute each sql statement
    statements = sqlparse.split(file_content)
    for stmt in statements:
        if verbose != "Select...":
            if verbose == "Y":
                st.code(f"""Statement executed: {stmt}
                """)
            session.sql(stmt).collect()
        if file_name == list(event_account_setup_list)[-1][0] and stmt == statements[-1]:
            st.success(f"Events Account configured 🎉")